In [1]:
import pandas as pd

In [2]:
acs = pd.read_csv('./acs_ny.csv')
acs.head()

,Acres,FamilyIncome,FamilyType,NumBedrooms,NumChildren,NumPeople,NumRooms,NumUnits,NumVehicles,NumWorkers,OwnRent,YearBuilt,HouseCosts,ElectricBill,FoodStamp,HeatingFuel,Insurance,Language
0,1-10,150,Married,4,1,3,9,Single detached,1,0,Mortgage,1950-1959,1800,90,No,Gas,2500,English
1,1-10,180,Female Head,3,2,4,6,Single detached,2,0,Rented,Before 1939,850,90,No,Oil,0,English
2,1-10,280,Female Head,4,0,2,8,Single detached,3,1,Mortgage,2000-2004,2600,260,No,Oil,6600,Other European
3,1-10,330,Female Head,2,1,2,4,Single detached,1,0,Rented,1950-1959,1800,140,No,Oil,0,English
4,1-10,330,Male Head,3,1,2,5,Single attached,1,0,Mortgage,Before 1939,860,150,No,Gas,660,Spanish


In [5]:
acs['ge150k'] =pd.cut(acs['FamilyIncome'],[0,150000,acs['FamilyIncome'].max()],labels=[0,1])
acs[['ge150k','FamilyIncome']]

,ge150k,FamilyIncome
0,0,150
1,0,180
2,0,280
3,0,330
4,0,330
5,0,480
6,0,520
7,0,550
8,0,1400
9,0,1400


In [6]:
acs.dtypes

Acres             object
FamilyIncome       int64
FamilyType        object
NumBedrooms        int64
NumChildren        int64
NumPeople          int64
NumRooms           int64
NumUnits          object
NumVehicles        int64
NumWorkers         int64
OwnRent           object
YearBuilt         object
HouseCosts         int64
ElectricBill       int64
FoodStamp         object
HeatingFuel       object
Insurance          int64
Language          object
ge150k          category
dtype: object

In [7]:
acs['ge150k_i'] =acs['ge150k'].astype(int)

In [8]:
acs['ge150k_i'].value_counts()

0    18294
1     4451
Name: ge150k_i, dtype: int64

In [9]:
import statsmodels.formula.api as smf

In [10]:
model =smf.logit('ge150k_i ~ HouseCosts + NumWorkers + OwnRent + NumBedrooms + FamilyType', data= acs)

In [11]:
results = model.fit()


Optimization terminated successfully.
         Current function value: 0.391651
         Iterations 7


In [14]:
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                             Results: Logit
========================================================================
Model:                 Logit              No. Iterations:     7.0000    
Dependent Variable:    ge150k_i           Pseudo R-squared:   0.208     
Date:                  2018-04-15 22:20   AIC:                17832.2108
No. Observations:      22745              BIC:                17896.4676
Df Model:              7                  Log-Likelihood:     -8908.1   
Df Residuals:          22737              LL-Null:            -11244.   
Converged:             1.0000             Scale:              1.0000    
------------------------------------------------------------------------
                         Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------------
Intercept               -5.8081   0.1199 -48.4563 0.0000 -6.0430 -5.5732
OwnRent[T.Outright]      1.8276   0.2081   8.7822 0.0000  1.4198  2.2355
OwnRent[T.Rented]       -0.8763   0.1013  -8.6468 0.0000 -1.0750 -0.6777
FamilyType[T.Male Head]  0.2874   0.1502   1.9126 0.0558 -0.0071  0.5818
FamilyType[T.Married]    1.3877   0.0879  15.7807 0.0000  1.2153  1.5601
HouseCosts               0.0007   0.0000  42.4535 0.0000  0.0007  0.0008
NumWorkers               0.5873   0.0262  22.3930 0.0000  0.5359  0.6387
NumBedrooms              0.2365   0.0169  13.9852 0.0000  0.2034  0.2697
========================================================================

"""

In [15]:
import numpy as np

In [16]:
np.exp(results.params)

Intercept                  0.003003
OwnRent[T.Outright]        6.219147
OwnRent[T.Rented]          0.416310
FamilyType[T.Male Head]    1.332901
FamilyType[T.Married]      4.005636
HouseCosts                 1.000731
NumWorkers                 1.799117
NumBedrooms                1.266852
dtype: float64

In [17]:
predictors = pd.get_dummies(acs[['HouseCosts','NumWorkers','OwnRent','NumBedrooms','FamilyType']],drop_first=True)


In [18]:
from sklearn import linear_model


In [19]:
lr = linear_model.LogisticRegression()

In [20]:
results = lr.fit(X=predictors, y=acs['ge150k_i'])

In [22]:
results.coef_ , results.intercept_

(array([[ 7.11127007e-04,  5.61096831e-01,  2.22827785e-01,
          1.20384525e+00, -7.16411565e-01,  3.42249546e-01,
          1.22562400e+00]]), array([-5.51245099]))

In [24]:
values = np.append(results.intercept_, results.coef_)
names= np.append('intercept', predictors.columns)
results =pd.DataFrame(values,index=names, columns=['coef'])

In [25]:
results['or'] = np.exp(results['coef'])
results.head()

,coef,or
intercept,-5.512451,0.004036
HouseCosts,0.000711,1.000711
NumWorkers,0.561097,1.752594
NumBedrooms,0.222828,1.249605
OwnRent_Outright,1.203845,3.332908


In [37]:
np.exp(0.014634*50)*0.657

1.365647587898324

In [27]:
3.337/3.289

1.0145941015506232

In [38]:
1.3656/(1.3656+1)

0.5772742644572202